In [7]:
import dolma

In [8]:
config_path = "../configs/cccc"
base_config = """
documents:
{paths}

dedupe:
  name: dedupe_url
  documents:
    attribute_name: dedupe_url
    key: $.metadata.url
  skip_empty: true


bloom_filter:
  file: /tmp/dedupe-url/cccc-{snapshot}/filter.bloom
  read_only: false
  estimated_doc_count: 100_000_000
  desired_false_positive_rate: 1e-06

processes: 188
work_dir:
  input: /tmp/dedupe-url/cccc-{snapshot}/input
  output: /tmp/dedupe-url/cccc-{snapshot}/output
"""

In [9]:
import boto3


def list_zst_files(bucket_name, prefix):
    s3_client = boto3.client("s3")
    paginator = s3_client.get_paginator("list_objects_v2")

    # Create a paginator to iterate through the objects
    operation_parameters = {"Bucket": bucket_name, "Prefix": prefix}
    page_iterator = paginator.paginate(**operation_parameters)

    zst_files = []

    for page in page_iterator:
        if "Contents" in page:
            for obj in page["Contents"]:
                if obj["Key"].endswith(".zst"):
                    zst_files.append(obj["Key"])

    return zst_files

In [10]:
from dolma.core.paths import glob_path, mkdir_p
import itertools

config_url_path = config_path + "/dedupe-url"
mkdir_p(config_url_path)

SKIP_SNAPSHOTS = [
    "CC-MAIN-2016-26",
    "CC-MAIN-2016-30",
    "CC-MAIN-2018-09",
    "CC-MAIN-2017-30",
    "CC-MAIN-2018-09",
    "CC-MAIN-2019-51",
    "CC-MAIN-2019-43",
    "CC-MAIN-2020-24",
    "CC-MAIN-2020-10",
    "CC-MAIN-2020-40",
    "CC-MAIN-2023-40",
]
commands = []

for snapshot_path in glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/*"):
    snapshot = snapshot_path.split("/")[-1]

    if snapshot in SKIP_SNAPSHOTS:
        print("Skipping", snapshot)
        continue

    bucket_name, prefix = snapshot_path.split("/", 3)[2:]
    paths = [f"    - s3://{bucket_name}/{path}" for path in list_zst_files(bucket_name, prefix)]
    print(snapshot_path, len(paths))
    config = base_config.format(paths="\n".join(paths), snapshot=snapshot)
    with open(config_url_path + f"/{snapshot}.yaml", "w") as f:
        f.write(config)

    commands.append(f"dolma -c {config_url_path.lstrip('../')}/{snapshot}.yaml dedupe")

with open(config_path + "/run-dedupe-url.sh", "w") as f:
    f.write("#!/bin/bash\n\n")
    f.write("set -ex\n\n")
    f.write("\n".join(commands))


s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-20 316
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-48 519
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-10 557
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-15 465
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-23 636
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-35 529
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-41 483
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-42 597
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-49 320
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-52 437
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-06 252
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-11 331
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-14 292
s3://ai2-llm

In [11]:
config_path = "../configs/cccc"
base_para_config = """
documents:
{paths}

dedupe:
  name: dedupe_para
  paragraphs:
    attribute_name: dedupe_para
    by_ngram:
      ngram_length: 20
      stride: 1
      overlap_threshold: 0.5
      skip_short_paragraphs: true
  skip_empty: true


bloom_filter:
  file: /tmp/dedupe-para/cccc-{snapshot}/filter.bloom
  read_only: false
  estimated_doc_count: 50_000_000_000
  desired_false_positive_rate: 1e-02

processes: 188
work_dir:
  input: /tmp/dedupe-para/cccc-{snapshot}/input
  output: /tmp/dedupe-para/cccc-{snapshot}/output
"""

In [12]:
from dolma.core.paths import glob_path, mkdir_p
import itertools

config_url_path = config_path + "/dedupe-para"
mkdir_p(config_url_path)

SKIP_SNAPSHOTS = [
    "CC-MAIN-2016-26",
    "CC-MAIN-2016-30",
    "CC-MAIN-2018-09",
    "CC-MAIN-2017-30",
    "CC-MAIN-2018-09",
    "CC-MAIN-2019-51",
    "CC-MAIN-2019-43",
    "CC-MAIN-2020-24",
    "CC-MAIN-2020-10",
    "CC-MAIN-2020-40",
    "CC-MAIN-2023-40",
]
commands = []

for snapshot_path in glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/*"):
    snapshot = snapshot_path.split("/")[-1]

    if snapshot in SKIP_SNAPSHOTS:
        print("Skipping", snapshot)
        continue

    bucket_name, prefix = snapshot_path.split("/", 3)[2:]
    paths = [f"    - s3://{bucket_name}/{path}" for path in list_zst_files(bucket_name, prefix)]
    print(snapshot_path, len(paths))
    config = base_para_config.format(paths="\n".join(paths), snapshot=snapshot)
    with open(config_url_path + f"/{snapshot}.yaml", "w") as f:
        f.write(config)

    commands.append(f"dolma -c {config_url_path.lstrip('../')}/{snapshot}.yaml dedupe")

with open(config_path + "/run-dedupe-para.sh", "w") as f:
    f.write("#!/bin/bash\n\n")
    f.write("set -ex\n\n")
    f.write("\n".join(commands))

s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-20 316
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-48 519
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-10 557
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-15 465
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-23 636
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-35 529
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-41 483
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-42 597
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-49 320
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-52 437
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-06 252
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-11 331
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-14 292
s3://ai2-llm

In [13]:
from dolma.core.paths import make_relative

all_snapshots = set(glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/*"))
counted_snapshots = set(
    f"s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/{snapshot_path.rsplit('/', 1)[-1]}"
    for snapshot_path in glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/attributes/dedupe_url/*")
)
to_skip = set([f"s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/{snapshot}" for snapshot in SKIP_SNAPSHOTS])

# print(len(all_snapshots))
# print(len(counted_snapshots))
# print(len(to_skip))
# print(len(all_snapshots - counted_snapshots - to_skip))


to_glob = []
for snapshot_path in sorted(all_snapshots - counted_snapshots - to_skip):
    snapshot = snapshot_path.split("/")[-1]
    bucket_name, prefix = snapshot_path.split("/", 3)[2:]
    all_zst = list_zst_files(bucket_name, prefix)

    nested = sum(1 for path in all_zst if "/warc/" in path)

    if nested > 0:
        print(f"  - s3://{bucket_name}/{prefix}/*/warc/*.zst")
    if nested < len(all_zst):
        print(f"  - s3://{bucket_name}/{prefix}/*.zst")

In [16]:
mixer_config = """
streams:
  - name: cccc-{snapshot}
    documents:{paths}
    attributes:
      - c4_v2
      - dedupe_url
      - ft_lang_id_1e2
      - gopher_v1
      - tokenizer_repetitions_v2r2
      - whitespace_tokenizer_v1
    output:
      max_size_in_bytes: 2_000_000_000
      path: s3://ai2-llm/pretraining-data/sources/cccc/v1/documents/{snapshot}
      min_text_length: 25   # matches wikipedia
    filter:
      include:
        # Only English
        - >-
          (.attributes.ft_lang_id_1e2__ft_lang_id_1e2__en != null) and
          (.attributes.ft_lang_id_1e2__ft_lang_id_1e2__en[0][2] >= 0.5)
      exclude:
        # Duplicated URLs
        - (.attributes.dedupe_url | length > 1)

        # C4 Rules
        - >-
          (.attributes.c4_v2__c4_v2__has_curly_brace != null) and
          (.attributes.c4_v2__c4_v2__has_curly_brace[0][2] > 0.5)
        - >-
          (.attributes.c4_v2__c4_v2__has_lorem_ipsum != null) and
          (.attributes.c4_v2__c4_v2__has_lorem_ipsum[0][2] > 0.5)
        - >-
          (.attributes.c4_v2__c4_v2__has_javascript != null) and
          (.attributes.c4_v2__c4_v2__has_javascript[0][2] > 0.5)

        # Gopher Rules
        - >-
          (.attributes.gopher_v1__gopher_v1__word_count != null) and
          (.attributes.gopher_v1__gopher_v1__word_count[0][2] < 50)
        - >-
          (.attributes.gopher_v1__gopher_v1__word_count != null) and
          (.attributes.gopher_v1__gopher_v1__word_count[0][2] > 100000)
        - >-
          (.attributes.gopher_v1__gopher_v1__median_word_length != null) and
          (.attributes.gopher_v1__gopher_v1__median_word_length[0][2] < 3)
        - >-
          (.attributes.gopher_v1__gopher_v1__median_word_length != null) and
          (.attributes.gopher_v1__gopher_v1__median_word_length[0][2] > 10)
        - >-
          (.attributes.gopher_v1__gopher_v1__symbol_to_word_ratio != null) and
          (.attributes.gopher_v1__gopher_v1__symbol_to_word_ratio[0][2] > 0.1)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_words_with_alpha_character != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_words_with_alpha_character[0][2] < 0.8)
        - >-
          (.attributes.gopher_v1__gopher_v1__required_word_count != null) and
          (.attributes.gopher_v1__gopher_v1__required_word_count[0][2] < 2)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_lines_starting_with_bullet_point != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_lines_starting_with_bullet_point[0][2] > 0.9)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_lines_ending_with_ellipsis != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_lines_ending_with_ellipsis[0][2] > 0.3)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_duplicate_lines != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_duplicate_lines[0][2] > 0.3)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_lines != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_lines[0][2] > 0.3)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_2gram != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_2gram[0][2] > 0.2)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_3gram != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_3gram[0][2] > 0.18)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_4gram != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_most_common_4gram[0][2] > 0.16)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_5grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_5grams[0][2] > 0.15)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_6grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_6grams[0][2] > 0.14)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_7grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_7grams[0][2] > 0.13)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_8grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_8grams[0][2] > 0.12)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_9grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_9grams[0][2] > 0.11)
        - >-
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_10grams != null) and
          (.attributes.gopher_v1__gopher_v1__fraction_of_characters_in_duplicate_10grams[0][2] > 0.10)

        # Remove repetitions
        - >-
          (.attributes.tokenizer_repetitions_v2r2__tokenizer_repetitions_v2r2__doc_max_score_repetition != null) and
          (.attributes.tokenizer_repetitions_v2r2__tokenizer_repetitions_v2r2__doc_max_score_repetition > 10)

work_dir:
  input: "/tmp/cccc-{snapshot}/mix/input"
  output: "/tmp/cccc-{snapshot}/mix/output"

processes: 100
"""

In [17]:
from dolma.core.paths import glob_path, mkdir_p
import itertools

config_url_path = config_path + "/mixer"
mkdir_p(config_url_path)

all_snapshots = set(glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/*"))
counted_snapshots = set(
    f"s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/{snapshot_path.rsplit('/', 1)[-1]}"
    for snapshot_path in glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/attributes/dedupe_url/*")
)
to_skip = set(
    [f"s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/{snapshot}" for snapshot in SKIP_SNAPSHOTS]
)

commands = []

for snapshot_path in glob_path("s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/*"):
    snapshot = snapshot_path.split("/")[-1]

    if snapshot in SKIP_SNAPSHOTS:
        print("Skipping", snapshot)
        continue

    bucket_name, prefix = snapshot_path.split("/", 3)[2:]
    paths = [f"          - s3://{bucket_name}/{path}" for path in list_zst_files(bucket_name, prefix)]
    print(snapshot_path, len(paths))
    config = mixer_config.format(paths="\n" + "\n".join(paths), snapshot=snapshot)
    with open(config_url_path + f"/{snapshot}.yaml", "w") as f:
        f.write(config)

    commands.append(f"dolma -c {config_url_path.lstrip('../')}/{snapshot}.yaml mix")

with open(config_path + "/run-mix.sh", "w") as f:
    f.write("#!/bin/bash\n\n")
    f.write("set -ex\n\n")
    f.write("\n".join(commands))

s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-20 316
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2013-48 519
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-10 557
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-15 465
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-23 636
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-35 529
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-41 483
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-42 597
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-49 320
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2014-52 437
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-06 252
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-11 331
s3://ai2-llm/pretraining-data/sources/cccc/v0/documents/CC-MAIN-2015-14 292
s3://ai2-llm